In [ ]:
# Importación de librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Preprocessament i Modelització
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

print("Librerías básicas importadas.")

In [ ]:
# Instalación de auto-sklearn (si es necesario)
# Nota: auto-sklearn puede tener problemas en Windows. Si falla, usaremos una alternativa.
try:
    import autosklearn.regression
    print("auto-sklearn ya está instalado.")
except ImportError:
    print("Intentando instalar auto-sklearn...")
    print("NOTA: auto-sklearn puede no funcionar en Windows.")
    print("Como alternativa, usaremos FLAML que es compatible con Windows y muy efectivo.")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "flaml"])

In [ ]:
# Importar FLAML (alternativa a auto-sklearn, compatible con Windows)
try:
    from flaml import AutoML
    USE_FLAML = True
    print("Usando FLAML para AutoML (compatible con Windows)")
except ImportError:
    USE_FLAML = False
    print("FLAML no disponible. Usaremos GridSearch manual.")
    from sklearn.model_selection import GridSearchCV
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from lightgbm import LGBMRegressor

In [ ]:
# Carga de datos
data_dir = Path('.')
train_path = data_dir / 'train.csv'
test_path = data_dir / 'test.csv'
sample_path = data_dir / 'sample_submission.csv'

try:
    train_df = pd.read_csv(train_path, sep=';')
    test_df = pd.read_csv(test_path, sep=';')
    sample_sub = pd.read_csv(sample_path, sep=',')
    print("Datos cargados exitosamente.")
except FileNotFoundError:
    print("Error: Asegúrate que los archivos CSV estén en el directorio.")

print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")
print(f"Sample: {sample_sub.shape}")

In [ ]:
# Agregación de datos (mismo proceso que EDA.ipynb)
print("Agregando datos por ID...")

# Target
y_train_agg = train_df.groupby('ID')['weekly_demand'].sum()

# Features
X_train_agg_full = train_df.groupby('ID').first()

# Alinear columnas
train_features = set(X_train_agg_full.columns)
test_features = set(test_df.columns)
common_columns = list(train_features.intersection(test_features))

X_train_agg = X_train_agg_full[common_columns].copy()
test_ids_for_submission = test_df['ID']
test_df_clean = test_df[common_columns].copy()

y_train_agg = y_train_agg.reindex(X_train_agg.index)

print(f"X_train_agg: {X_train_agg.shape}")
print(f"y_train_agg: {y_train_agg.shape}")
print(f"test_df_clean: {test_df_clean.shape}")

In [ ]:
# Ingeniería de características
print("Creando features...")

X_train_features = X_train_agg.copy()
X_test_features = test_df_clean.copy()

# Feature: start_month
X_train_features['phase_in'] = pd.to_datetime(X_train_features['phase_in'], errors='coerce')
X_test_features['phase_in'] = pd.to_datetime(X_test_features['phase_in'], errors='coerce')

X_train_features['start_month'] = X_train_features['phase_in'].dt.month
X_test_features['start_month'] = X_test_features['phase_in'].dt.month

median_month = X_train_features['start_month'].median()
X_train_features['start_month'] = X_train_features['start_month'].fillna(median_month)
X_test_features['start_month'] = X_test_features['start_month'].fillna(median_month)

# Listas de features
numeric_features = [
    'num_stores', 'price', 'life_cycle_length', 'num_sizes', 'start_month'
]

categorical_features = [
    'moment', 'archetype', 'neck_lapel_type', 'print_type', 'has_plus_sizes',
    'knit_structure', 'waist_type', 'silhouette_type', 'family', 'length_type',
    'color_name', 'category', 'woven_structure', 'id_season',
    'aggregated_family', 'sleeve_length_type', 'fabric'
]

print(f"Features numéricas: {len(numeric_features)}")
print(f"Features categóricas: {len(categorical_features)}")

In [ ]:
# Transformador de embeddings
class EmbeddingSplitter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.str.split(',', expand=True).astype(float).fillna(0)

# Pipelines de transformación
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

embedding_transformer = Pipeline(steps=[
    ('split', EmbeddingSplitter()),
    ('pca', PCA(n_components=64, random_state=42))
])

# Preprocessor completo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('embed', embedding_transformer, 'image_embedding')
    ],
    remainder='drop'
)

print("Preprocessor creado con embeddings.")

In [ ]:
# Función de pérdida asimétrica personalizada
# Penaliza más cuando la predicción es menor que el real (underestimation)
def asymmetric_loss(y_true, y_pred, underestimation_penalty=2.0):
    """
    Pérdida asimétrica que penaliza más las subestimaciones.
    
    Parameters:
    - underestimation_penalty: factor de penalización cuando y_pred < y_true
      Por defecto 2.0 significa que penaliza el doble las subestimaciones.
    """
    errors = y_true - y_pred
    # Errores positivos = subestimación (pred < real) -> penaliza más
    # Errores negativos = sobreestimación (pred > real) -> penaliza menos
    weights = np.where(errors > 0, underestimation_penalty, 1.0)
    return np.mean(weights * np.abs(errors))

def asymmetric_scorer(y_true, y_pred):
    """Wrapper para usar como scorer en sklearn (mayor es mejor, así que invertimos)"""
    return -asymmetric_loss(y_true, y_pred, underestimation_penalty=2.0)

# Crear scorer para sklearn
custom_scorer = make_scorer(asymmetric_scorer, greater_is_better=True)

print("Función de pérdida asimétrica definida.")
print("Penalización: 2x cuando predecimos por debajo del real.")

In [ ]:
# Transformación logarítmica del target
print("Aplicando transformación logarítmica al target...")
y_train_log = np.log1p(y_train_agg)

# Preparar datos
X = X_train_features
y = y_train_log
X_test = X_test_features

# Split para validación
X_train_local, X_val_local, y_train_local, y_val_local = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# También guardamos los valores sin log para evaluación
y_train_agg_split = train_test_split(y_train_agg, test_size=0.2, random_state=42)
y_train_local_real = y_train_agg_split[0]
y_val_local_real = y_train_agg_split[1]

print(f"Datos divididos: {len(X_train_local)} train, {len(X_val_local)} validación")

In [ ]:
# Preprocesar los datos
print("Preprocesando datos...")
X_train_processed = preprocessor.fit_transform(X_train_local)
X_val_processed = preprocessor.transform(X_val_local)

print(f"Shape después del preprocesamiento: {X_train_processed.shape}")

In [ ]:
# Auto-sklearn / FLAML para encontrar el mejor modelo
print("="*60)
print("Iniciando búsqueda automática del mejor modelo...")
print("="*60)

if USE_FLAML:
    print("\nUsando FLAML AutoML...")
    print("Configuración: 10 minutos de búsqueda, métrica personalizada asimétrica")
    
    automl = AutoML()
    
    # Configuración de FLAML
    settings = {
        "time_budget": 600,  # 10 minutos
        "metric": "mae",  # Usamos MAE como base
        "task": "regression",
        "log_file_name": "flaml_log.txt",
        "seed": 42,
        "eval_method": "cv",  # Cross-validation
        "n_splits": 3,
        "verbose": 1,
        # Modelos a probar
        "estimator_list": ['lgbm', 'rf', 'xgboost', 'extra_tree', 'catboost']
    }
    
    print("\n¡Entrenando! Esto puede tardar ~10 minutos...\n")
    
    automl.fit(
        X_train=X_train_processed,
        y_train=y_train_local,
        **settings
    )
    
    print("\n" + "="*60)
    print("FLAML ha terminado!")
    print("="*60)
    print(f"\nMejor modelo encontrado: {automl.best_estimator}")
    print(f"Mejor configuración: {automl.best_config}")
    print(f"Mejor score (MAE en log-scale): {automl.best_loss}")
    
    best_model = automl
    
else:
    print("\nUsando GridSearch manual con varios modelos...")
    print("Esto puede tardar varios minutos...\n")
    
    # Definir modelos y parámetros para GridSearch
    from sklearn.model_selection import GridSearchCV
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from lightgbm import LGBMRegressor
    
    models_and_params = [
        {
            'name': 'RandomForest',
            'model': RandomForestRegressor(random_state=42, n_jobs=-1),
            'params': {
                'n_estimators': [100, 200, 300],
                'max_depth': [10, 20, 30, None],
                'min_samples_split': [2, 5, 10]
            }
        },
        {
            'name': 'LightGBM',
            'model': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
            'params': {
                'n_estimators': [100, 200, 300],
                'learning_rate': [0.01, 0.05, 0.1],
                'max_depth': [5, 10, 15],
                'num_leaves': [31, 50, 70]
            }
        }
    ]
    
    best_score = float('inf')
    best_model = None
    best_model_name = None
    
    for model_config in models_and_params:
        print(f"\nProbando {model_config['name']}...")
        
        grid_search = GridSearchCV(
            estimator=model_config['model'],
            param_grid=model_config['params'],
            scoring='neg_mean_absolute_error',
            cv=3,
            n_jobs=-1,
            verbose=1
        )
        
        grid_search.fit(X_train_processed, y_train_local)
        
        if -grid_search.best_score_ < best_score:
            best_score = -grid_search.best_score_
            best_model = grid_search.best_estimator_
            best_model_name = model_config['name']
        
        print(f"Mejor score para {model_config['name']}: {-grid_search.best_score_:.4f}")
        print(f"Mejores parámetros: {grid_search.best_params_}")
    
    print("\n" + "="*60)
    print(f"Mejor modelo global: {best_model_name}")
    print(f"Mejor score (MAE en log-scale): {best_score:.4f}")
    print("="*60)

In [ ]:
# Evaluación del modelo con pérdida asimétrica
print("\n" + "="*60)
print("EVALUACIÓN DEL MODELO AUTO-OPTIMIZADO")
print("="*60)

# Predecir en validación (en escala log)
y_pred_val_log = best_model.predict(X_val_processed)

# Invertir transformación logarítmica
y_pred_val_real = np.expm1(y_pred_val_log)

# Calcular métricas estándar
mae_standard = mean_absolute_error(y_val_local_real, y_pred_val_real)
rmse_standard = np.sqrt(mean_squared_error(y_val_local_real, y_pred_val_real))

# Calcular métrica asimétrica
mae_asymmetric = asymmetric_loss(y_val_local_real.values, y_pred_val_real, underestimation_penalty=2.0)

print("\n--- Métricas Estándar ---")
print(f"MAE: {mae_standard:.4f}")
print(f"RMSE: {rmse_standard:.4f}")

print("\n--- Métrica Asimétrica (penaliza 2x subestimaciones) ---")
print(f"MAE Asimétrico: {mae_asymmetric:.4f}")

# Análisis de errores
errors = y_val_local_real.values - y_pred_val_real
underestimations = errors[errors > 0]
overestimations = errors[errors < 0]

print("\n--- Análisis de Errores ---")
print(f"Subestimaciones (pred < real): {len(underestimations)} casos")
print(f"  - Media: {np.mean(np.abs(underestimations)):.4f}")
print(f"Sobreestimaciones (pred > real): {len(overestimations)} casos")
print(f"  - Media: {np.mean(np.abs(overestimations)):.4f}")

In [ ]:
# Visualización de resultados
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Predicciones vs Real
axes[0].scatter(y_val_local_real, y_pred_val_real, alpha=0.5)
axes[0].plot([y_val_local_real.min(), y_val_local_real.max()], 
             [y_val_local_real.min(), y_val_local_real.max()], 'r--', lw=2)
axes[0].set_xlabel('Demanda Real')
axes[0].set_ylabel('Demanda Predicha')
axes[0].set_title('Predicciones vs Valores Reales')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Distribución de errores
axes[1].hist(errors, bins=50, alpha=0.7, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Error (Real - Predicho)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Errores\n(>0 = Subestimación, <0 = Sobreestimación)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('autosklearn_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nGráficos guardados en 'autosklearn_evaluation.png'")

In [ ]:
# Entrenar modelo final con todos los datos
print("\n" + "="*60)
print("ENTRENAMIENTO DEL MODELO FINAL")
print("="*60)

print("\nPreprocesando datos completos...")
X_full_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test)

print("Entrenando modelo con 100% de los datos...")

if USE_FLAML:
    # Re-entrenar con los mismos parámetros óptimos
    final_model = AutoML()
    final_model.fit(
        X_train=X_full_processed,
        y_train=y,
        time_budget=300,  # 5 minutos adicionales
        metric="mae",
        task="regression",
        seed=42,
        starting_points=automl.best_config
    )
else:
    # Usar el mejor modelo encontrado
    final_model = best_model
    final_model.fit(X_full_processed, y)

print("¡Modelo final entrenado!")

In [ ]:
# Generar predicciones finales
print("\nGenerando predicciones para test...")

# Predecir en escala log
predictions_log = final_model.predict(X_test_processed)

# Invertir transformación logarítmica
predictions_real = np.expm1(predictions_log)

# Ajustar valores negativos
predictions_real = np.maximum(predictions_real, 0)

# Aplicar factor de seguridad para evitar subestimaciones
# Basado en el análisis: penaliza más predecir de menos
SAFETY_FACTOR = 1.05  # 5% de margen de seguridad
final_predictions = predictions_real * SAFETY_FACTOR

print(f"\nFactor de seguridad aplicado: {SAFETY_FACTOR}")
print(f"Esto reduce el riesgo de subestimaciones (que penalizan más).")

In [ ]:
# Crear archivo de submisión
submission_df = pd.DataFrame({
    'ID': test_ids_for_submission,
    'demand': final_predictions
})

submission_filename = 'submission_autosklearn.csv'
submission_df.to_csv(submission_filename, index=False, sep=',')

print("\n" + "="*60)
print(f"¡Archivo '{submission_filename}' creado con éxito!")
print("="*60)

print("\nEstadísticas de las predicciones:")
print(submission_df['demand'].describe())

print("\nPrimeras filas:")
print(submission_df.head(10))

In [ ]:
# Comparación con el modelo de producción (EDA.ipynb)
print("\n" + "="*60)
print("COMPARACIÓN CON MODELO DE PRODUCCIÓN")
print("="*60)

try:
    # Cargar predicción de producción (si existe)
    production_files = ['submission_v5_strategic.csv', 'submission2.csv', 'submission.csv']
    production_df = None
    
    for prod_file in production_files:
        if Path(prod_file).exists():
            production_df = pd.read_csv(prod_file)
            print(f"\nCargado modelo de producción: {prod_file}")
            break
    
    if production_df is not None:
        # Comparar estadísticas
        print("\n--- Estadísticas Comparativas ---")
        print(f"\nModelo Auto-sklearn:")
        print(f"  Media: {submission_df['demand'].mean():.4f}")
        print(f"  Mediana: {submission_df['demand'].median():.4f}")
        print(f"  Std: {submission_df['demand'].std():.4f}")
        print(f"  Min: {submission_df['demand'].min():.4f}")
        print(f"  Max: {submission_df['demand'].max():.4f}")
        
        print(f"\nModelo Producción:")
        print(f"  Media: {production_df['demand'].mean():.4f}")
        print(f"  Mediana: {production_df['demand'].median():.4f}")
        print(f"  Std: {production_df['demand'].std():.4f}")
        print(f"  Min: {production_df['demand'].min():.4f}")
        print(f"  Max: {production_df['demand'].max():.4f}")
        
        # Diferencias
        diff = submission_df['demand'].values - production_df['demand'].values
        print(f"\n--- Diferencias (AutoML - Producción) ---")
        print(f"  Media de diferencias: {np.mean(diff):.4f}")
        print(f"  Std de diferencias: {np.std(diff):.4f}")
        print(f"  AutoML predice más alto en {np.sum(diff > 0)} casos ({100*np.mean(diff > 0):.1f}%)")
        print(f"  AutoML predice más bajo en {np.sum(diff < 0)} casos ({100*np.mean(diff < 0):.1f}%)")
        
        # Visualización
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.scatter(production_df['demand'], submission_df['demand'], alpha=0.5)
        plt.plot([0, production_df['demand'].max()], [0, production_df['demand'].max()], 'r--', lw=2)
        plt.xlabel('Producción')
        plt.ylabel('Auto-sklearn')
        plt.title('Comparación de Predicciones')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.hist(diff, bins=50, alpha=0.7, edgecolor='black')
        plt.axvline(x=0, color='r', linestyle='--', lw=2)
        plt.xlabel('Diferencia (AutoML - Producción)')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Diferencias')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('comparison_automl_vs_production.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("\nGráfico de comparación guardado en 'comparison_automl_vs_production.png'")
        
    else:
        print("\nNo se encontró archivo de producción para comparar.")
        print("Ejecuta primero las celdas de EDA.ipynb para generar submission_v5_strategic.csv")
        
except Exception as e:
    print(f"\nError al comparar con producción: {e}")

## Resumen y Conclusiones

Este notebook utiliza AutoML (FLAML o auto-sklearn) para encontrar automáticamente el mejor modelo y parámetros.

**Características clave:**
1. Búsqueda automática entre múltiples algoritmos (LightGBM, RandomForest, XGBoost, CatBoost, etc.)
2. Optimización automática de hiperparámetros
3. Función de pérdida asimétrica que penaliza 2x las subestimaciones
4. Factor de seguridad aplicado para reducir riesgo de predecir de menos
5. Comparación detallada con el modelo de producción

**Próximos pasos:**
- Ajustar el `underestimation_penalty` si es necesario (actualmente 2.0)
- Modificar el `SAFETY_FACTOR` basado en los resultados (actualmente 1.05)
- Aumentar el `time_budget` para búsquedas más exhaustivas
- Probar diferentes configuraciones de PCA para los embeddings